<a href="https://colab.research.google.com/github/loveHeath0105/BIG_DATA_PRACTICAL/blob/main/week_3_pratical01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Installation

In [ ]:
%pip install duckdb leafmap

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 515.2/515.2 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.7/213.7 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 49.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.6/108.6 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 40.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 765.5/765.5 kB 32.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.2/194.2 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.4/72.4 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 43.9 MB/s eta 0:00:00


In [ ]:
import duckdb
import leafmap

In [ ]:
url = "https://open.gishub.org/data/duckdb/nyc_data.db.zip"
leafmap.download_file(url, unzip=True)

Downloading...
From: https://open.gishub.org/data/duckdb/nyc_data.db.zip
To: /content/nyc_data.db.zip
100%|██████████| 8.60M/8.60M [00:00<00:00, 73.7MB/s]


Extracting files...


'/content/nyc_data.db.zip'

In [ ]:
con = duckdb.connect('nyc_data.db')

In [ ]:
con.install_extension('spatial')
con.load_extension('spatial')

In [ ]:
con.sql("SHOW TABLES;")

┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ nyc_census_blocks   │
│ nyc_homicides       │
│ nyc_neighborhoods   │
│ nyc_streets         │
│ nyc_subway_stations │
└─────────────────────┘

##QUESTION
# question 1
**What is the area of the 'West Village' neighborhood? **  
(Hint: The area is given in square meters. To get an area in hectares, divide by 10000. To get an area in acres, divide by 4047.)

In [ ]:
neighbours_df=con.sql("""
SELECT *
FROM nyc_neighborhoods
LIMIT 5;
""").df()
neighbours_df.head()




,BORONAME,NAME,geom
0,Brooklyn,Bensonhurst,"[5, 4, 41, 0, 0, 0, 0, 0, 54, 71, 14, 73, 198,..."
1,Manhattan,East Village,"[5, 4, 152, 0, 0, 0, 0, 0, 35, 215, 14, 73, 13..."
2,Manhattan,West Village,"[5, 4, 91, 0, 0, 0, 0, 0, 161, 95, 14, 73, 212..."
3,The Bronx,Throggs Neck,"[5, 4, 141, 0, 0, 0, 0, 0, 128, 232, 17, 73, 1..."
4,The Bronx,Wakefield-Williamsbridge,"[5, 4, 126, 0, 0, 0, 0, 0, 83, 85, 17, 73, 17,..."


In [ ]:
con.sql("""
SELECT NAME, ST_AREA(geom)/10000 AS area_hectares
FROM nyc_neighborhoods
WHERE NAME='West Village';
""")

┌──────────────┬────────────────────┐
│     NAME     │   area_hectares    │
│   varchar    │       double       │
├──────────────┼────────────────────┤
│ West Village │ 104.46145296485956 │
└──────────────┴────────────────────┘

# QUESTION 2
What is the geometry type of ‘Pelham St’? The length?

In [ ]:
con.sql("""
SELECT ST_GeometryType(geom), ST_LENGTH(geom)
FROM nyc_streets
WHERE name='Pelham St';
""")


┌───────────────────────┬───────────────────┐
│ st_geometrytype(geom) │  st_length(geom)  │
│     geometry_type     │      double       │
├───────────────────────┼───────────────────┤
│ MULTILINESTRING       │ 50.32314951660229 │
└───────────────────────┴───────────────────┘

# question 3
What is the GeoJSON representation of the 'Broad St' subway station?

In [ ]:
con.sql("""
SELECT ST_AsGeoJSON(geom)
FROM nyc_subway_stations
WHERE name='Broad St';

""")

┌──────────────────────────────────────────────────────────────────────┐
│                          st_asgeojson(geom)                          │
│                                 json                                 │
├──────────────────────────────────────────────────────────────────────┤
│ {"type":"Point","coordinates":[583571.9059213118,4506714.341192182]} │
└──────────────────────────────────────────────────────────────────────┘

# QUESTION 4
**What is the total length of streets (in kilometers) in New York City?**  
 (Hint: The units of measurement of the spatial data are meters, there are 1000 meters in a kilometer.)

In [ ]:
con.sql("""
SELECT SUM(ST_Length(geom))/1000 AS length_km
FROM nyc_streets;
""")


┌────────────────────┐
│     length_km      │
│       double       │
├────────────────────┤
│ 10418.904717199997 │
└────────────────────┘

#QUESTION 5
**What is the area of Manhattan in acres?**  
(Hint: both nyc_census_blocks and nyc_neighborhoods have a boroname in them.)

In [ ]:
con.sql("""
SELECT SUM(ST_Area(geom))/4047 AS area_acres
FROM nyc_neighborhoods
WHERE boroname='Manhattan';
""")

┌───────────────────┐
│    area_acres     │
│      double       │
├───────────────────┤
│ 13965.32012239119 │
└───────────────────┘

# QUESTION 6
What is the most westerly subway station?


In [ ]:
con.sql("""
SELECT NAME, ST_X(geom)
FROM nyc_subway_stations
ORDER BY ST_X(geom) ASC
LIMIT 5;
""")

┌─────────────────┬───────────────────┐
│      NAME       │    st_x(geom)     │
│     varchar     │      double       │
├─────────────────┼───────────────────┤
│ Tottenville     │ 563292.1172580556 │
│ Atlantic        │ 563901.9011614178 │
│ Nassau          │ 564500.8510479712 │
│ Richmond Valley │ 565312.0938333038 │
│ Pleasant Plains │ 566327.1555864002 │
└─────────────────┴───────────────────┘

# QUESTION 7
**How long is 'Columbus Cir' (aka Columbus Circle)?**

In [ ]:
con.sql("""
SELECT NAME,ST_Length(geom) AS lenghth
FROM nyc_streets
WHERE NAME = 'Columbus Cir';
""")


┌──────────────┬───────────────────┐
│     NAME     │      lenghth      │
│   varchar    │      double       │
├──────────────┼───────────────────┤
│ Columbus Cir │ 308.3419936909855 │
└──────────────┴───────────────────┘

# QUESTION 8
What is the length of streets in New York City, summarized by type?

In [ ]:
con.sql("""
SELECT SUM(ST_Length(geom)) AS length, type
FROM nyc_streets
GROUP BY type
ORDER BY length DESC;

""")



┌────────────────────┬──────────────────────────────────────────────────┐
│       length       │                       TYPE                       │
│       double       │                     varchar                      │
├────────────────────┼──────────────────────────────────────────────────┤
│   8629870.33786606 │ residential                                      │
│  403622.4781263628 │ motorway                                         │
│  360394.8790513027 │ tertiary                                         │
│   294261.419479668 │ motorway_link                                    │
│  276264.3038979258 │ secondary                                        │
│  166936.3716044583 │ unclassified                                     │
│  135034.2330179469 │ primary                                          │
│  71798.48783780965 │ footway                                          │
│ 28337.635038596007 │ service                                          │
│  20353.58198260764 │ trunk          